# 1.2 Run Red teaming agent in the Cloud

## Notes
- You must use a Foundry project for this feature. A hub based project isn't supported.
- Running the AI Red Teaming Agent in the cloud currently only supports Azure OpenAI model deployments in your Azure AI Foundry project as a target.

## Initial setup
If this is your first time running evaluations or AI red teaming runs on your Azure AI Foundry project, you might need to do a few additional setup steps. You can find these steps in the file `/infra/CloudRedTeaming-Guide.ipynb`.

## Load environment variables


In [ ]:
#!az login

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("../infra/credentials.env", override=True)

endpoint = os.environ["AZURE_PJ_PROJECT_ENDPOINT"] 
model_endpoint = os.environ["AZURE_PJ_MODEL_ENDPOINT"]
model_api_key= os.environ["AZURE_PJ_MODEL_API_KEY"]
model_deployment_name = os.environ["AZURE_PJ_MODEL_NAME"]

print("Endpoint:", endpoint)
print("Model Endpoint:", model_endpoint) 
print("Model Deployment Name:", model_deployment_name)
print("Model API Key:", model_api_key)   

## Create an AI red teaming run

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    RedTeam,
    AzureOpenAIModelConfiguration,
    AttackStrategy,
    RiskCategory,
)

with AIProjectClient(
  endpoint=endpoint,
  credential=DefaultAzureCredential(),
) as project_client:

    # Create target configuration for testing an Azure OpenAI model
    target_config = AzureOpenAIModelConfiguration(
        model_deployment_name= "RedTeaming-OpenAI/" + model_deployment_name,
    )

    print("Target Configuration:", target_config.model_deployment_name)

    # Instantiate the AI Red Teaming Agent
    red_team_agent = RedTeam(
        attack_strategies=[AttackStrategy.BASE64],
        risk_categories=[RiskCategory.VIOLENCE],
        display_name="red-team-cloud-run", 
        target=target_config,
    )


    # Create and run the red teaming scan
    red_team_response = project_client.red_teams.create(red_team=red_team_agent,  headers={
        "endpoint": endpoint,
        "api-version": "2024-12-01-preview",   
    },)


## Get an AI red teaming run

In [ ]:
# Use the name returned by the create operation for the get call
get_red_team_response = project_client.red_teams.get(name=red_team_response.name)
print(f"Red Team scan status: {get_red_team_response.status}")

## List all AI red teaming runs

In [ ]:
for scan in project_client.red_teams.list():
  print(f"Found scan: {scan.name}, Status: {scan.status}")